# Sistema recomendador  de Ciber-Seguridad basado en raiting por usuario

In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# PATH
path_datos_raiting = '/home/krlozz/Escritorio/Investigacion-ciberataques-sistema-recomendacion/datos-prueba/raiting_25.csv'
path_datos_ataques = '/home/krlozz/Escritorio/Investigacion-ciberataques-sistema-recomendacion/datos-prueba/datos_valores_25.csv'

In [2]:
raitings = pd.read_csv(path_datos_raiting, sep=',', names=['user_id','item_id','rating'])
raitings.head()

,user_id,item_id,rating
0,1,18,2
1,1,17,1
2,1,16,2
3,1,1,3
4,1,12,3


In [3]:
items = pd.read_csv(path_datos_ataques)
items.head()

,item_id,anomalia,descripcion,criticidad,recomendacion
0,18,Improper Control of Generation of Code ('Code ...,The software constructs all or part of a code ...,Medium,a
1,17,Improper Restriction of XML External Entity Re...,The software processes an XML document that ca...,None,b
2,16,Unrestricted Upload of File with Dangerous Type,The software allows the attacker to upload or ...,Medium,c
3,1,Improper Restriction of Operations within the ...,The software performs operations on a memory b...,High,d
4,12,Out-of-bounds Write,"The software writes data past the end, or befo...",High,e


In [4]:
ataques_raitings = pd.merge(raitings, items, on='item_id')
ataques_raitings.head()

,user_id,item_id,rating,anomalia,descripcion,criticidad,recomendacion
0,1,18,2,Improper Control of Generation of Code ('Code ...,The software constructs all or part of a code ...,Medium,a
1,2,18,3,Improper Control of Generation of Code ('Code ...,The software constructs all or part of a code ...,Medium,a
2,3,18,1,Improper Control of Generation of Code ('Code ...,The software constructs all or part of a code ...,Medium,a
3,1,17,1,Improper Restriction of XML External Entity Re...,The software processes an XML document that ca...,None,b
4,2,17,3,Improper Restriction of XML External Entity Re...,The software processes an XML document that ca...,None,b


In [57]:
media_ataques_raitings = pd.DataFrame(ataques_raitings.groupby(['name'])['rating'].mean())
media_ataques_raitings.head()

,rating
name,
CWE-119: Improper Restriction of Operations within the Bounds of a Memory Buffer,1.666667
CWE-125: Out-of-bounds Read,2.000000
CWE-190: Integer Overflow or Wraparound,1.666667
CWE-200: Information Exposure,2.666667
CWE-20: Improper Input Validation,1.666667


In [58]:
media_ataques_raitings['numero_de_rating'] = ataques_raitings.groupby(['name'])['rating'].count()
media_ataques_raitings.head()

,rating,numero_de_rating
name,,
CWE-119: Improper Restriction of Operations within the Bounds of a Memory Buffer,1.666667,3
CWE-125: Out-of-bounds Read,2.000000,3
CWE-190: Integer Overflow or Wraparound,1.666667,3
CWE-200: Information Exposure,2.666667,3
CWE-20: Improper Input Validation,1.666667,3


In [59]:
# pivot_table tabla de estadistica
# pivot_table resumen datos de una tabla mas extensa, suma, promedios, etc
# pivot_table procesamiento de datos, organizan y reorganizan informacion util
matriz_usuario_ataque = ataques_raitings.pivot_table(index='user_id', columns='name', values='rating')
matriz_usuario_ataque.head()

name,CWE-119: Improper Restriction of Operations within the Bounds of a Memory Buffer,CWE-125: Out-of-bounds Read,CWE-190: Integer Overflow or Wraparound,CWE-200: Information Exposure,CWE-20: Improper Input Validation,CWE-22: Improper Limitation of a Pathname to a Restricted Directory ('Path Traversal'),CWE-269: Improper Privilege Management,CWE-287: Improper Authentication,CWE-295: Improper Certificate Validation,CWE-352: Cross-Site Request Forgery (CSRF),...,CWE-502: Deserialization of Untrusted Data,CWE-611: Improper Restriction of XML External Entity Reference,CWE-732: Incorrect Permission Assignment for Critical Resource,CWE-772: Missing Release of Resource after Effective Lifetime,CWE-787: Out-of-bounds Write,CWE-78: Improper Neutralization of Special Elements used in an OS Command ('OS Command Injection'),CWE-798: Use of Hard-coded Credentials,CWE-79: Improper Neutralization of Input During Web Page Generation ('Cross-site Scripting'),CWE-89: Improper Neutralization of Special Elements used in an SQL Command ('SQL Injection'),CWE-94: Improper Control of Generation of Code ('Code Injection')
user_id,,,,,,,,,,,,,,,,,,,,,
1,3,1,2,3,3,3,2,3,1,2,...,2,1,3,3,3,3,3,3,3,2
2,1,2,1,2,1,3,3,3,2,3,...,2,3,1,2,1,3,1,2,3,3
3,1,3,2,3,1,2,1,2,1,3,...,3,1,2,3,2,3,2,3,3,1


In [60]:
media_ataques_raitings.sort_values('numero_de_rating', ascending=False).head()

,rating,numero_de_rating
name,,
CWE-119: Improper Restriction of Operations within the Bounds of a Memory Buffer,1.666667,3
CWE-434: Unrestricted Upload of File with Dangerous Type,2.000000,3
CWE-89: Improper Neutralization of Special Elements used in an SQL Command ('SQL Injection'),3.000000,3
CWE-79: Improper Neutralization of Input During Web Page Generation ('Cross-site Scripting'),2.666667,3
CWE-798: Use of Hard-coded Credentials,2.000000,3


In [61]:
user_rating = matriz_usuario_ataque['CWE-20: Improper Input Validation']
user_rating.head()

user_id
1    3
2    1
3    1
Name: CWE-20: Improper Input Validation, dtype: int64

In [62]:
recomendacion_similar = matriz_usuario_ataque.corrwith(user_rating)
recomendacion_similar.head()

name
CWE-119: Improper Restriction of Operations within the Bounds of a Memory Buffer    1.000000
CWE-125: Out-of-bounds Read                                                        -0.866025
CWE-190: Integer Overflow or Wraparound                                             0.500000
CWE-200: Information Exposure                                                       0.500000
CWE-20: Improper Input Validation                                                   1.000000
dtype: float64

In [63]:
correlacion_recomendaciones = pd.DataFrame(recomendacion_similar, columns=['correlation'])
correlacion_recomendaciones.dropna(inplace=True)
correlacion_recomendaciones.head()

,correlation
name,
CWE-119: Improper Restriction of Operations within the Bounds of a Memory Buffer,1.000000
CWE-125: Out-of-bounds Read,-0.866025
CWE-190: Integer Overflow or Wraparound,0.500000
CWE-200: Information Exposure,0.500000
CWE-20: Improper Input Validation,1.000000


In [64]:
correlacion_recomendaciones = correlacion_recomendaciones.join(media_ataques_raitings['numero_de_rating'],how='left', lsuffix='_left', rsuffix='_right')
correlacion_recomendaciones.head()

,correlation,numero_de_rating
name,,
CWE-119: Improper Restriction of Operations within the Bounds of a Memory Buffer,1.000000,3
CWE-125: Out-of-bounds Read,-0.866025,3
CWE-190: Integer Overflow or Wraparound,0.500000,3
CWE-200: Information Exposure,0.500000,3
CWE-20: Improper Input Validation,1.000000,3


In [65]:
# numero de recomendaciones por usuario
correlacion_recomendaciones[correlacion_recomendaciones['numero_de_rating'] > 2].sort_values(by='correlation', ascending=False).head()


,correlation,numero_de_rating
name,,
CWE-119: Improper Restriction of Operations within the Bounds of a Memory Buffer,1.000000,3
CWE-20: Improper Input Validation,1.000000,3
CWE-798: Use of Hard-coded Credentials,0.866025,3
CWE-787: Out-of-bounds Write,0.866025,3
CWE-732: Incorrect Permission Assignment for Critical Resource,0.866025,3
